In [1]:
import pandas as pd
import folium
import geopandas
from shapely.geometry import Point
import zipfile
import os
import shutil
import json
import time

In [2]:
scenario_name = "1"
path_to_zip_file = '/home/ai6644/Malmo/Tools/Simulation results/Lolland/TRB/drt/base-rerun DRT instead of RAIL/'+\
                   scenario_name+'/log.zip'

In [3]:
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    try:
        shutil.rmtree('data/zip')
    except (FileNotFoundError, OSError) as e:
        pass
    os.mkdir('data/zip')
    zip_ref.extractall('data/zip')
path = 'data/zip/' + os.listdir('data/zip')[0] + '/trip_dump.json'

In [4]:
fp = open(path)
tdump = json.load(fp)
fp.close()
persons = tdump.get('person')

In [5]:
drt_trips = []
pt_trips = []
car_trips = []
car_people = []
pt_people = []
drt_people = []
for person in persons:
    trips = person.get('actual_trips')
    for trip in trips:
        modes = [leg['mode'] for leg in trip.get('legs')]
        if 'CAR' in modes:
            car_trips.append(trip)
            car_people.append(person)
        elif 'DRT' in modes:
            drt_trips.append(trip)
            drt_people.append(person)
        else:
            pt_trips.append(trip)
            pt_people.append(person)

In [6]:
print('CAR trips {}'.format(len(car_trips)))
print('PT trips {}'.format(len(pt_trips)))
print('DRT trips {}'.format(len(drt_trips)))

CAR trips 11
PT trips 1261
DRT trips 1257


In [7]:
car_lines = []
no_location = set()
for trips in car_trips:
    legs = trips['legs']
    
    lat = legs[0]['start_coord']['lat']
    lon = legs[0]['start_coord']['lon']
    slatlon = (lat,lon)

    lat = legs[-1]['end_coord']['lat']
    lon = legs[-1]['end_coord']['lon']
    elatlon = (lat,lon)
    car_lines.append((slatlon,elatlon))

In [8]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l,t,p in zip(car_lines, car_trips, car_people):
    str_time = time.strftime('%H:%M:%S', time.gmtime(t['legs'][0]['start_time']))
    str_id = str(p['id'])
    folium.PolyLine(l[0:2], weight=3, tooltip=str(l)+" "+str_time+" "+str_id).add_to(m)
m.save('data/car_trips.html')

In [9]:
pt_lines = []
no_location = set()
for trips in pt_trips:
    legs = trips['legs']
    
    lat = legs[0]['start_coord']['lat']
    lon = legs[0]['start_coord']['lon']
    slatlon = (lat,lon)

    lat = legs[-1]['end_coord']['lat']
    lon = legs[-1]['end_coord']['lon']
    elatlon = (lat,lon)
    pt_lines.append((slatlon,elatlon))

In [10]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l,t,p in zip(pt_lines, pt_trips, pt_people):
    str_time = time.strftime('%H:%M:%S', time.gmtime(t['legs'][0]['start_time']))
    str_id = str(p['id'])
    folium.PolyLine(l[0:2], weight=3, tooltip=str(l)+" "+str_time+" "+str_id).add_to(m)
m.save('data/pt_trips.html')

In [11]:
drt_lines = []
no_location = set()
for trips in drt_trips:
    legs = trips['legs']
    
    lat = legs[0]['start_coord']['lat']
    lon = legs[0]['start_coord']['lon']
    slatlon = (lat,lon)

    lat = legs[-1]['end_coord']['lat']
    lon = legs[-1]['end_coord']['lon']
    elatlon = (lat,lon)
    drt_lines.append((slatlon,elatlon))

In [12]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l,t,p in zip(drt_lines, drt_trips, drt_people):
    str_time = time.strftime('%H:%M:%S', time.gmtime(t['legs'][0]['start_time']))
    str_id = str(p['id'])
    folium.PolyLine(l[0:2], weight=3, tooltip=str(l)+" "+str_time+" "+str_id).add_to(m)
m.save('data/drt_trips.html')

# Trips to/from Nakskov

In [13]:
import math
def is_close(coord1, coord2):
    if math.isclose(coord1['lat'], coord2['lat'], abs_tol=0.001) and math.isclose(coord1['lon'], coord2['lon'], abs_tol=0.001):
        return True
    else:
        return False

In [14]:
nakskov_st = []
nakskov_et = []
for trip in car_trips:
    if is_close(trip['legs'][0]['start_coord'], {'lat': 54.83346, 'lon': 11.13814}):
        nakskov_st.append(trip)
    elif is_close(trip['legs'][0]['end_coord'], {'lat': 54.83346, 'lon': 11.13814}):
        nakskov_et.append(trip)

In [15]:
import time
time.strftime('%H:%M:%S', time.gmtime(12345))

'03:25:45'

In [16]:
lines = []
no_location = set()
for trip in nakskov_st:
    legs = trip['legs']
    
    lat = legs[0]['start_coord']['lat']
    lon = legs[0]['start_coord']['lon']
    slatlon = (lat,lon)

    lat = legs[-1]['end_coord']['lat']
    lon = legs[-1]['end_coord']['lon']
    elatlon = (lat,lon)
    lines.append((slatlon,elatlon))
    
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for l,t in zip(lines, nakskov_st):
    str_time = time.strftime('%H:%M:%S', time.gmtime(t['legs'][0]['start_time']))
    folium.PolyLine(l[0:2], weight=2, tooltip='nakskov-> {}'.format(str_time)).add_to(m)

lines = []
no_location = set()
for trips in nakskov_et:
    legs = trips['legs']
    
    lat = legs[0]['start_coord']['lat']
    lon = legs[0]['start_coord']['lon']
    slatlon = (lat,lon)

    lat = legs[-1]['end_coord']['lat']
    lon = legs[-1]['end_coord']['lon']
    elatlon = (lat,lon)
    lines.append((slatlon,elatlon))
    
for l,t in zip(lines, nakskov_et):
    str_time = time.strftime('%H:%M:%S', time.gmtime(t['legs'][0]['start_time']))
    folium.PolyLine(l[0:2], weight=2, tooltip='nakskov<- {}'.format(str_time)).add_to(m)
m.save('data/nakskov_trips.html')

In [17]:
nakskov = [trip for trip in car_trips
           if is_close(trip['legs'][0]['end_coord'], {'lat': 54.83346, 'lon': 11.13814}) and 
              is_close(trip['legs'][0]['start_coord'], {'lat': 54.69996, 'lon': 11.39579})]